In [0]:
%tensorflow_version 1.x

# ReLU(Rectified Linear Unit)
### 이해하기  
  * 먼저 NN(2): MLP 코드를 이용하여, layer를 9개로 쌓아 코드를 돌려보자

In [0]:
import tensorflow as tf
import numpy as np

x_data = np.array([[0,0], [0,1], [1,0], [1,1]], dtype = np.float32)
y_data = np.array([[0],   [1],   [1],   [0]], dtype = np.float32)



X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)



# weight와 bias 생성
W0 = tf.Variable(tf.random_uniform([2, 5], -1.0, 1.0), name = "weight0") # input layer
W1 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight1") #
W2 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight2")
W3 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight3")
W4 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight4")
W5 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight5")
W6 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight6")
W7 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight7")
W8 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight8")
W9 = tf.Variable(tf.random_uniform([5, 5], -1.0, 1.0), name = "weight9") #
W10 = tf.Variable(tf.random_uniform([5, 1], -1.0, 1.0), name = "weight10") # output layer

b0 = tf.Variable(tf.zeros([5]), name = "bias0") # input layer
b1 = tf.Variable(tf.zeros([5]), name = "bias1") #
b2 = tf.Variable(tf.zeros([5]), name = "bias2")
b3 = tf.Variable(tf.zeros([5]), name = "bias3")
b4 = tf.Variable(tf.zeros([5]), name = "bias4")
b5 = tf.Variable(tf.zeros([5]), name = "bias5")
b6 = tf.Variable(tf.zeros([5]), name = "bias6")
b7 = tf.Variable(tf.zeros([5]), name = "bias7")
b8 = tf.Variable(tf.zeros([5]), name = "bias8")
b9 = tf.Variable(tf.zeros([5]), name = "bias9") #
b10 = tf.Variable(tf.zeros([5]), name = "bias10") # output layer

L1 = tf.sigmoid(tf.matmul(X, W0) + b0) # input layer
L2 = tf.sigmoid(tf.matmul(L1, W1) + b1) #
L3 = tf.sigmoid(tf.matmul(L2, W2) + b2)
L4 = tf.sigmoid(tf.matmul(L3, W3) + b3)
L5 = tf.sigmoid(tf.matmul(L4, W4) + b4)
L6 = tf.sigmoid(tf.matmul(L5, W5) + b5)
L7 = tf.sigmoid(tf.matmul(L6, W6) + b6)
L8 = tf.sigmoid(tf.matmul(L7, W7) + b7)
L9 = tf.sigmoid(tf.matmul(L8, W8) + b8)
L10 = tf.sigmoid(tf.matmul(L9, W9) + b9) # 
hypothesis = tf.sigmoid(tf.matmul(L10, W10) + b10) # output layer



cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)



predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32)) 



with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for step in range(10001):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    
  h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict = {X: x_data, Y: y_data})
  print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


Hypothesis:  [[0.5000002  0.5000002  0.5000002  0.5000002  0.5000002 ]
 [0.5000001  0.5000001  0.5000001  0.5000001  0.5000001 ]
 [0.5000001  0.5000001  0.5000001  0.5000001  0.5000001 ]
 [0.50000006 0.50000006 0.50000006 0.50000006 0.50000006]] 
Correct:  [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]] 
Accuracy:  0.5


* 분명 NN(2)에서 설명하길, layer를 많이 쌓으면(DNN) 성능이 좋아진다고 했는데, 9개로 쌓다보니 정확도가 떨어졌다. 왜일까?

## Vanishing Gradient
* NN(3): Backpropagation에서의 설명을 잠시 다시 꺼내와보자  
  ![image](https://user-images.githubusercontent.com/38516906/76195070-dc285280-622a-11ea-90fe-d1d1481dccb6.png)
  * 우리는 backpropagation에서 chain rule을 사용한다고 했다.
  * 여기서 y는 input layer가 아니었다면, 다른 유닛을 통과하고 sigmoid 함수를 통과했을 것이다  
    = 정확한 y값은 알지 못해도, sigmoid 특성 때문에 0~1 사이의 값을 갖는 것은 예상할 수 있음
  * 그럼 y를 0.01로 가정하고 봤을 때 df/dx = df/dg * 0.01이 됨
  * 즉 여기서 말하고 싶은 점은,  
    **sigmoid 함수를 지나면서 그 값이 1보다 작게되며,  
    경우에 따라서는 굉장이 0에 가까운 값들이 계속 곱해지게 된다는 것**
    -> 굉장히 작은 값을 가지게 됨  
  * 따라서 layer가 깊어질수록 그 영향도를 찾기 힘들며, 여측하기 힘들다는 결론이 나온다.  
  전문용어로는 **Vanishing gradient**
<br><br>

## ReLU 등장
* Hilton 교수님은 이러한 vanishing gradient의 현상에는 sigmoid에 있다는 것을 밝혀내셨음
  * sigmoid 함수 때문에 1보다 큰 값을 가지지 못하기 때문
  * 그래서 layer가 깊으지면 오히려 그 값이 작아져 영향을 찾기 힘들어짐
* 그리고 그 대처방안 함수가 바로, ReLU(Rectified Linear Unit)
![image](https://user-images.githubusercontent.com/38516906/76267549-809fa880-62ae-11ea-9e39-5479817b03ca.png)
  * 말만 어렵지 정말 단순한 형태이다
    * 입력값이 0보다 작음: non-activate로 꺼버림
    * 입력값이 0보다 큼: 그 값 그대로 반환
  * 사용법도 간단하다. 우리가 sigmoid를 사용했던 곳에 sigmoid 대신 ReLU를 사용하면 된다.
    ```
    L1 = tf.sigmoid(tf.matmul(X, W1) + b1)
    
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    ```
<br><br>

### 그 외의 함수들
정말 다양한 activation function이 있으니 확인해보자.
  ![image](https://user-images.githubusercontent.com/38516906/76267658-da07d780-62ae-11ea-86be-9856df7afa8e.png)
